In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer
from scipy.stats import boxcox

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
x = pd.read_csv('Summary of Weather.csv')

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,8,18,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
x.sample(5)

,STA,Date,Precip,WindGustSpd,MaxTemp,MinTemp,MeanTemp,Snowfall,PoorWeather,YR,...,FB,FTI,ITH,PGT,TSHDSBRSGF,SD3,RHX,RHN,RVG,WTE
98912,50302,30-07-1945,T,NaN,30.000000,21.111111,25.555556,0,1,45,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
21772,11604,30-11-1942,57.15,NaN,25.555556,23.333333,24.444444,0,1,42,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
69408,32601,29-09-1944,0,NaN,33.333333,25.555556,29.444444,0,1,44,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
101245,50401,16-11-1943,T,NaN,31.111111,22.222222,26.666667,0,1,43,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
72161,32702,10-05-1944,0,NaN,40.000000,26.111111,33.333333,0,NaN,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
x.shape

(119040, 31)

In [5]:
x.describe()

,STA,WindGustSpd,MaxTemp,MinTemp,MeanTemp,YR,MO,DA,DR,SPD,...,FT,FB,FTI,ITH,PGT,SD3,RHX,RHN,RVG,WTE
count,119040.000000,532.000000,119040.000000,119040.000000,119040.000000,119040.000000,119040.000000,119040.000000,533.000000,532.000000,...,0.0,0.0,0.0,0.0,525.000000,0.0,0.0,0.0,0.0,0.0
mean,29659.435795,37.774534,27.045111,17.789511,22.411631,43.805284,6.726016,15.797530,26.998124,20.396617,...,NaN,NaN,NaN,NaN,12.085333,NaN,NaN,NaN,NaN,NaN
std,20953.209402,10.297808,8.717817,8.334572,8.297982,1.136718,3.425561,8.794541,15.221732,5.560371,...,NaN,NaN,NaN,NaN,5.731328,NaN,NaN,NaN,NaN,NaN
min,10001.000000,18.520000,-33.333333,-38.333333,-35.555556,40.000000,1.000000,1.000000,2.000000,10.000000,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,11801.000000,29.632000,25.555556,15.000000,20.555556,43.000000,4.000000,8.000000,11.000000,16.000000,...,NaN,NaN,NaN,NaN,8.500000,NaN,NaN,NaN,NaN,NaN
50%,22508.000000,37.040000,29.444444,21.111111,25.555556,44.000000,7.000000,16.000000,32.000000,20.000000,...,NaN,NaN,NaN,NaN,11.600000,NaN,NaN,NaN,NaN,NaN
75%,33501.000000,43.059000,31.666667,23.333333,27.222222,45.000000,10.000000,23.000000,34.000000,23.250000,...,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,NaN,NaN,NaN
max,82506.000000,75.932000,50.000000,34.444444,40.000000,45.000000,12.000000,31.000000,78.000000,41.000000,...,NaN,NaN,NaN,NaN,23.900000,NaN,NaN,NaN,NaN,NaN


In [6]:

print (x.isnull().sum()/x.shape[0] * 100)

STA              0.000000
Date             0.000000
Precip           0.000000
WindGustSpd     99.553091
MaxTemp          0.000000
MinTemp          0.000000
MeanTemp         0.000000
Snowfall         0.976983
PoorWeather     71.239079
YR               0.000000
MO               0.000000
DA               0.000000
PRCP             1.622984
DR              99.552251
SPD             99.553091
MAX              0.398185
MIN              0.393145
MEA              0.418347
SNF              0.976983
SND             95.326781
FT             100.000000
FB             100.000000
FTI            100.000000
ITH            100.000000
PGT             99.558972
TSHDSBRSGF      71.239079
SD3            100.000000
RHX            100.000000
RHN            100.000000
RVG            100.000000
WTE            100.000000
dtype: float64


In [8]:
#Commented line to check if that column's null percentage is below 70

cols = [col for col in x.columns if (x[col].isnull().sum()/x.shape[0] * 100 < 70)]
x_trimmed = x[cols]

#STA is more of station code, lets drop it for the moment
x_trimmed = x_trimmed.drop(['STA'], axis=1)

print ('Legitimate columns after dropping null columns: %s' % x_trimmed.shape[1])

Legitimate columns after dropping null columns: 14


In [9]:
x_trimmed.isnull().sum()

Date           0
Precip         0
MaxTemp        0
MinTemp        0
MeanTemp       0
Snowfall    1163
YR             0
MO             0
DA             0
PRCP        1932
MAX          474
MIN          468
MEA          498
SNF         1163
dtype: int64

In [10]:
x_trimmed.sample(5)

,Date,Precip,MaxTemp,MinTemp,MeanTemp,Snowfall,YR,MO,DA,PRCP,MAX,MIN,MEA,SNF
65023,14-04-1943,0,33.333333,19.444444,26.111111,0,43,4,14,0,92.0,67.0,79.0,0
9712,23-06-1944,0.254,29.444444,22.777778,26.111111,0,44,6,23,0.01,85.0,73.0,79.0,0
105424,05-11-1945,T,28.333333,18.888889,23.333333,0,45,11,5,T,83.0,66.0,74.0,0
67185,15-12-1944,0,26.666667,21.111111,23.888889,0,44,12,15,0,80.0,70.0,75.0,0
55462,02-11-1942,16.51,25.000000,21.111111,23.333333,0,42,11,2,0.65,77.0,70.0,74.0,0


In [11]:
x_trimmed.dtypes

Date         object
Precip       object
MaxTemp     float64
MinTemp     float64
MeanTemp    float64
Snowfall     object
YR            int64
MO            int64
DA            int64
PRCP         object
MAX         float64
MIN         float64
MEA         float64
SNF          object
dtype: object

In [12]:
#Looks like some columns needs to be converted to numeric field

x_trimmed['Snowfall'] = pd.to_numeric(x_trimmed['Snowfall'], errors='coerce')
x_trimmed['SNF'] = pd.to_numeric(x_trimmed['SNF'], errors='coerce')
x_trimmed['PRCP'] = pd.to_numeric(x_trimmed['PRCP'], errors='coerce')
x_trimmed['Precip'] = pd.to_numeric(x_trimmed['Precip'], errors='coerce')

x_trimmed['Date'] = pd.to_datetime(x_trimmed['Date'])

In [13]:
#Fill remaining null values. FOr the moment lts perform ffill

x_trimmed.fillna(method='ffill', inplace=True)
x_trimmed.fillna(method='bfill', inplace=True)

x_trimmed.isnull().sum()
#Well no more NaN and null values to worry about

Date        0
Precip      0
MaxTemp     0
MinTemp     0
MeanTemp    0
Snowfall    0
YR          0
MO          0
DA          0
PRCP        0
MAX         0
MIN         0
MEA         0
SNF         0
dtype: int64

In [16]:
print (x_trimmed.dtypes)
print ('--'*30)
x_trimmed.sample(3)

Date        datetime64[ns]
Precip             float64
MaxTemp            float64
MinTemp            float64
MeanTemp           float64
Snowfall           float64
YR                   int64
MO                   int64
DA                   int64
PRCP               float64
MAX                float64
MIN                float64
MEA                float64
SNF                float64
dtype: object
------------------------------------------------------------


,Date,Precip,MaxTemp,MinTemp,MeanTemp,Snowfall,YR,MO,DA,PRCP,MAX,MIN,MEA,SNF
10389,1943-07-02,0.00,30.555556,23.333333,26.666667,0.0,43,2,7,0.00,87.0,74.0,80.0,0.0
51264,1945-04-29,3.81,28.888889,23.888889,26.666667,0.0,45,4,29,0.15,84.0,75.0,80.0,0.0
11435,1945-12-19,0.00,29.444444,23.888889,26.666667,0.0,45,12,19,0.00,85.0,75.0,80.0,0.0


In [33]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
# create dataset
X, y = make_blobs(n_samples=1000)
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(670, 2) (330, 2) (670,) (330,)


In [34]:
from sklearn.linear_model import LinearRegression  
regressor= LinearRegression()  
regressor.fit(X_train, y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [35]:
y_pred= regressor.predict(X_test)
print(y_pred)

[0.902847   1.16719169 0.98230527 0.85842153 1.45178249 0.9782191
 0.95092904 0.64573413 0.841211   1.09203386 1.32991741 0.80596728
 0.97717839 1.18394397 0.82362125 0.96989444 0.95751269 1.17744431
 1.01694267 1.24018834 0.90551046 0.77885208 1.23971367 1.11194746
 1.13125004 0.90058946 0.8775756  1.03715164 0.99770677 0.8931151
 1.08885586 0.82279315 1.00082121 1.11522594 1.0305852  1.14131721
 0.98218928 1.09313047 1.04201589 0.93704137 0.8997406  1.31927634
 1.06632504 1.19990033 1.05666517 0.85250425 1.10224212 1.01330739
 0.84583743 1.06113255 0.92875969 1.16011758 0.7499689  1.28889545
 0.750004   1.0314586  0.81513016 0.9364457  0.96265364 0.9112783
 0.72986254 0.99118714 1.01358339 1.28176093 0.80612541 1.0220449
 1.03554057 0.90204955 1.10330025 0.54728021 1.1789533  1.06225939
 1.1481457  1.05994113 0.91350412 1.371189   0.8088387  0.94638742
 0.93386929 1.06478017 0.79819713 1.22606155 1.23505353 0.9848655
 0.74542121 1.01883613 1.01798384 0.84613221 1.09155106 1.13310724


In [44]:

print(regressor.coef_)

[-0.08289674  0.13739879]


In [45]:
print(regressor.intercept_)

0.46519633059955545


In [46]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.030741600918477663